# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 3 2022 1:29PM,243389,10,0085922962,ISDIN Corporation,Released
1,Aug 3 2022 1:29PM,243389,10,0085922963,ISDIN Corporation,Released
2,Aug 3 2022 1:29PM,243389,10,0085922964,ISDIN Corporation,Released
3,Aug 3 2022 1:29PM,243389,10,0085922990,ISDIN Corporation,Released
4,Aug 3 2022 1:29PM,243389,10,0085922989,ISDIN Corporation,Released
5,Aug 3 2022 1:29PM,243389,10,0085922988,ISDIN Corporation,Released
6,Aug 3 2022 1:29PM,243389,10,0085922991,ISDIN Corporation,Released
7,Aug 3 2022 1:26PM,243387,20,ALU0007382,Alumier Labs Inc.,Released
8,Aug 3 2022 1:25PM,243388,10,0085922808,ISDIN Corporation,Released
9,Aug 3 2022 1:25PM,243388,10,0085922809,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,243386,Executing,1
39,243386,Released,1
40,243387,Released,1
41,243388,Released,8
42,243389,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243383,NaN,10.0
243386,1.0,1.0
243387,NaN,1.0
243388,NaN,8.0
243389,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243270,0.0,1.0
243294,0.0,2.0
243304,0.0,2.0
243306,0.0,7.0
243307,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243270,0,1
243294,0,2
243304,0,2
243306,0,7
243307,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243270,0,1
1,243294,0,2
2,243304,0,2
3,243306,0,7
4,243307,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243270,,1
1,243294,,2
2,243304,,2
3,243306,,7
4,243307,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation
7,Aug 3 2022 1:26PM,243387,20,Alumier Labs Inc.
8,Aug 3 2022 1:25PM,243388,10,ISDIN Corporation
16,Aug 3 2022 1:11PM,243386,20,Alumier Labs Inc.
18,Aug 3 2022 12:38PM,243383,10,ISDIN Corporation
28,Aug 3 2022 11:55AM,243382,10,Beach Products Inc
29,Aug 3 2022 11:51AM,243381,10,Emerginnova
31,Aug 3 2022 11:32AM,243379,10,Eywa Pharma Inc.
45,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc."
51,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation,,7
1,Aug 3 2022 1:26PM,243387,20,Alumier Labs Inc.,,1
2,Aug 3 2022 1:25PM,243388,10,ISDIN Corporation,,8
3,Aug 3 2022 1:11PM,243386,20,Alumier Labs Inc.,1,1
4,Aug 3 2022 12:38PM,243383,10,ISDIN Corporation,,10
5,Aug 3 2022 11:55AM,243382,10,Beach Products Inc,,1
6,Aug 3 2022 11:51AM,243381,10,Emerginnova,,2
7,Aug 3 2022 11:32AM,243379,10,Eywa Pharma Inc.,3,11
8,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc.",,6
9,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.,7,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation,7,
1,Aug 3 2022 1:26PM,243387,20,Alumier Labs Inc.,1,
2,Aug 3 2022 1:25PM,243388,10,ISDIN Corporation,8,
3,Aug 3 2022 1:11PM,243386,20,Alumier Labs Inc.,1,1
4,Aug 3 2022 12:38PM,243383,10,ISDIN Corporation,10,
5,Aug 3 2022 11:55AM,243382,10,Beach Products Inc,1,
6,Aug 3 2022 11:51AM,243381,10,Emerginnova,2,
7,Aug 3 2022 11:32AM,243379,10,Eywa Pharma Inc.,11,3
8,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc.",6,
9,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.,10,7


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation,7,
1,Aug 3 2022 1:26PM,243387,20,Alumier Labs Inc.,1,
2,Aug 3 2022 1:25PM,243388,10,ISDIN Corporation,8,
3,Aug 3 2022 1:11PM,243386,20,Alumier Labs Inc.,1,1
4,Aug 3 2022 12:38PM,243383,10,ISDIN Corporation,10,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation,7.0,NaN
1,Aug 3 2022 1:26PM,243387,20,Alumier Labs Inc.,1.0,NaN
2,Aug 3 2022 1:25PM,243388,10,ISDIN Corporation,8.0,NaN
3,Aug 3 2022 1:11PM,243386,20,Alumier Labs Inc.,1.0,1.0
4,Aug 3 2022 12:38PM,243383,10,ISDIN Corporation,10.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation,7.0,0.0
1,Aug 3 2022 1:26PM,243387,20,Alumier Labs Inc.,1.0,0.0
2,Aug 3 2022 1:25PM,243388,10,ISDIN Corporation,8.0,0.0
3,Aug 3 2022 1:11PM,243386,20,Alumier Labs Inc.,1.0,1.0
4,Aug 3 2022 12:38PM,243383,10,ISDIN Corporation,10.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4866843,152.0,8.0
12,243362,1.0,0.0
15,729964,13.0,0.0
16,243354,0.0,1.0
18,243325,1.0,0.0
19,486704,2.0,2.0
20,973507,14.0,11.0
21,1216598,4.0,1.0
22,243345,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4866843,152.0,8.0
1,12,243362,1.0,0.0
2,15,729964,13.0,0.0
3,16,243354,0.0,1.0
4,18,243325,1.0,0.0
5,19,486704,2.0,2.0
6,20,973507,14.0,11.0
7,21,1216598,4.0,1.0
8,22,243345,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,152.0,8.0
1,12,1.0,0.0
2,15,13.0,0.0
3,16,0.0,1.0
4,18,1.0,0.0
5,19,2.0,2.0
6,20,14.0,11.0
7,21,4.0,1.0
8,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,152.0
1,12,Released,1.0
2,15,Released,13.0
3,16,Released,0.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Executing,8.0,0.0,0.0,1.0,0.0,2.0,11.0,1.0,0.0
Released,152.0,1.0,13.0,0.0,1.0,2.0,14.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Executing,8.0,0.0,0.0,1.0,0.0,2.0,11.0,1.0,0.0
1,Released,152.0,1.0,13.0,0.0,1.0,2.0,14.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Executing,8.0,0.0,0.0,1.0,0.0,2.0,11.0,1.0,0.0
1,Released,152.0,1.0,13.0,0.0,1.0,2.0,14.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()